# Deep Sepsis Neural Network

In [3]:
from easy_sql.io import Session as session
from easy_sql.utils import read_sql_text
from sklearn.preprocessing import Imputer, StandardScaler
import numpy as np
from keras import Model
from keras.layers import Dense, Dropout, Input, GaussianNoise
from sklearn.metrics import roc_auc_score, accuracy_score
from keras.callbacks import EarlyStopping
import pandas as pd
import pickle
np.random.seed(7)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
dsn = 'muscedw'
sql_file_path = 'data/sepsis_NN.sql'
model_file_path = 'models/sepsis_nn.bundle'

# get data
s = session(dsn)
sql = read_sql_text(sql_file_path)
print('getting data from', dsn, sql)
data = s.get_data(sql)

time_col = 'AVAILABLE_TIME'
id_col = 'PAT_ENC_CSN_ID'
y_cols = ['MetSIRS4_8', 'MetSIRS4_24', 'MetSIRS4_48', 'MetSIRS4_4hr_8', 'MetSIRS4_4hr_24', 'MetSIRS4_4hr_48', 'MS870',
          'MS871', 'MS872', 'Sepsis_DRG']
non_x_cols = [time_col] + [id_col] + y_cols
x_cols = [col for col in data.columns if col not in non_x_cols]

# initialize imputer and scaler
imp = Imputer(strategy='median')
scaler = StandardScaler()

def input_fun(data, fit=False):
    # replace missing values with imputed ones
    temp_data = data[x_cols].replace(-99999, np.nan)
    if fit:
        print('fitting imputer and scaler')
        temp_data_imp = imp.fit_transform(temp_data)
        features = scaler.fit_transform(temp_data_imp)
    else:
        temp_data_imp = imp.transform(temp_data)
        features = scaler.transform(temp_data_imp)
    return features


# extract features
x = input_fun(data, fit=True)
y = np.array(data[y_cols])

# split the trainig and test data
split = .75
cut = int(split * x.shape[0])
x_train = x[:cut, :]
y_train = y[:cut, :]
x_test = x[cut:, :]
y_test = y[cut:, :]

n_input_dims = x_train.shape[1]
n_outputs = y_train.shape[1]


C:\Python36\Lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


getting data from muscedw 
SELECT TOP 1000000
       iplv. [PAT_ENC_CSN_ID]
	   ,iplv.[AVAILABLE_TIME]
      ,[AGE]
      ,[AdmissionsSixMonths]
      ,[MaxPreviousLOS]
      ,[ICUSixMonths]
      ,[PCPSixMonths]
      ,[SurgerySixMonths]
      ,[VenilatorSixMonths]
      ,[GLUCOSE, WHOLE BLOOD]
      ,[HEMOLYSIS INDEX]
      ,[SODIUM]
      ,[POTASSIUM]
      ,[GLUCOSE]
      ,[CREATININE]
      ,[CHLORIDE]
      ,[CALCIUM]
      ,[CO2 CONTENT (BICARBONATE)]
      ,[UREA NITROGEN, BLOOD (BUN)]
      ,[ANION GAP]
      ,[HEMATOCRIT]
      ,[HEMOGLOBIN]
      ,[PLATELET COUNT]
      ,[RED BLOOD CELL COUNT]
      ,[MEAN CORPUSCULAR HEMOGLOBIN]
      ,[MEAN CORPUSCULAR HEMOGLOBIN CONC]
      ,[MEAN CORPUSCULAR VOLUME]
      ,[WHITE BLOOD CELL COUNT]
      ,[RED CELL DISTRIBUTION WIDTH]
      ,[MEAN PLATELET VOLUME]
      ,[ICTERIC INDEX]
      ,[MAGNESIUM]
      ,[NUCLEATED RED BLOOD CELLS]
      ,[PHOSPHORUS (PO4)]
      ,[EGFR]
      ,[BILIRUBIN, TOTAL]
      ,[TOTAL PROTEIN]
      ,[AL

ProgrammingError: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'StatisticalModels.dbo.InpatientPipelineLabsVitals'. (208) (SQLExecDirectW)") [SQL: "\nSELECT TOP 1000000\n       iplv. [PAT_ENC_CSN_ID]\n\t   ,iplv.[AVAILABLE_TIME]\n      ,[AGE]\n      ,[AdmissionsSixMonths]\n      ,[MaxPreviousLOS]\n      ,[ICUSixMonths]\n      ,[PCPSixMonths]\n      ,[SurgerySixMonths]\n      ,[VenilatorSixMonths]\n      ,[GLUCOSE, WHOLE BLOOD]\n      ,[HEMOLYSIS INDEX]\n      ,[SODIUM]\n      ,[POTASSIUM]\n      ,[GLUCOSE]\n      ,[CREATININE]\n      ,[CHLORIDE]\n      ,[CALCIUM]\n      ,[CO2 CONTENT (BICARBONATE)]\n      ,[UREA NITROGEN, BLOOD (BUN)]\n      ,[ANION GAP]\n      ,[HEMATOCRIT]\n      ,[HEMOGLOBIN]\n      ,[PLATELET COUNT]\n      ,[RED BLOOD CELL COUNT]\n      ,[MEAN CORPUSCULAR HEMOGLOBIN]\n      ,[MEAN CORPUSCULAR HEMOGLOBIN CONC]\n      ,[MEAN CORPUSCULAR VOLUME]\n      ,[WHITE BLOOD CELL COUNT]\n      ,[RED CELL

In [ ]:
data[x_cols].head()

In [ ]:
print(x_cols) # input columns

In [ ]:
print(y_cols) # output columns

fit a deep feed foreward neural net, using dropout and noise layers to reduce over fitting

In [ ]:
esm = EarlyStopping(patience=2)
# trains a multiple output model
inputs = Input(shape=(n_input_dims, ))
noise = GaussianNoise(1.5)(inputs)
dense1 = Dense(100, activation='relu')(noise)
dropout1 = Dropout(.5)(dense1)
dense2 = Dense(100, activation='relu')(dropout1)
dropout2 = Dropout(.5)(dense2)
outputs = Dense(n_outputs, activation='sigmoid')(dropout2)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=50, batch_size=1000, validation_data=(x_test, y_test), verbose=2, callbacks=[esm],
          shuffle=True)


Check the model performance

In [ ]:
model.summary()
# generate predictions
preds_train = model.predict(x_train)
preds_test = model.predict(x_test)

# prints auc off all the output nodes
perf = []
for i in range(n_outputs):
    test_auc = roc_auc_score(y_test[:, i], preds_test[:, i])
    train_auc = roc_auc_score(y_train[:, i], preds_train[:, i])
    perf.append([y_cols[i] + ' train auc: ' + str(train_auc) + ' test auc: ' + str(test_auc)])

print(perf)

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
print('complete')

#### Feature Contributions
SIRS 4 within 48 Hourse


In [ ]:
i = 0
print(y_cols[i])
n = NeuralNetDescriber(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test, model=model, feature_names=x_cols, scaler=scaler, output_names=y_cols)
n.preds_test = preds_test
n.preds_train = preds_train
n.n_dims = x_train.shape[1]
n.n_obs = x_train.shape[0]
n.plot_decision_boundary(x_name = 'BLOOD PRESSURE (SYSTOLIC)',y_name = 'BLOOD PRESSURE (DIASTOLIC)', cut_off=.15, index=i)
n.plot_decision_boundary(x_name = 'RESPIRATIONS', y_name = 'PULSE', cut_off=.15, index=i)

In [ ]:
i = 6
for i ,j in enumerate(x_cols):
    print('variable: ' + j)
    try:
        n.plot_variable_sensitivity(j, index=i)
    except:
        pass

In [ ]:
from rnner.plot_methods import plot_auc_curve

for i, j in enumerate(y_cols):
    print('model ' + j)
    plot_auc_curve(y_train[:,i], preds_train[:,i], y_test[:,i], preds_test[:,i])


In [ ]:
from rnner.plot_methods import plot_density

for i, j in enumerate(y_cols):
    print('model ' + j)
    plot_density(preds_train[:,i], y_train[:,i])

In [ ]:
n.plot_decision_boundary(x_name = 'RESPIRATIONS', y_name = 'TEMPERATURE', cut_off=.15, index=5)
n.plot_decision_boundary(x_name = 'RESPIRATIONS', y_name = 'TEMPERATURE', cut_off=.15, index=5)
n.plot_decision_boundary(x_name = 'RESPIRATIONS', y_name = 'TEMPERATURE', cut_off=.15, index=5)